# ДЗ 2: Сравнение теггеров
Анастасия Добрынина, БКЛ-211

In [1]:
from collections import Counter
import json
import pandas as pd
from sklearn.metrics import accuracy_score
from nltk import word_tokenize
from string import punctuation
punctuation += '—'+'–'

import stanza
stanza.download('ru')

stnz = stanza.Pipeline('ru')

from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

from pymystem3 import Mystem

m = Mystem()

2023-10-07 12:34:59 INFO: Downloading default packages for language: ru (Russian) ...
2023-10-07 12:35:00 INFO: File exists: C:\Users\79998\stanza_resources\ru\default.zip
2023-10-07 12:35:04 INFO: Finished downloading models and saved to C:\Users\79998\stanza_resources.
2023-10-07 12:35:04 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-07 12:35:06 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2023-10-07 12:35:06 INFO: Use device: cpu
2023-10-07 12:35:06 INFO: Loading: tokenize
2023-10-07 12:35:06 INFO: Loading: pos
2023-10-07 12:35:06 INFO: Loading: lemma
2023-10-07 12:35:06 INFO: Loading: depparse
2023-10-07 12:35:06 INFO: Loading: ner
2023-10-07 12:35:07 INFO: Done loading processors!


### Проблемы автоматической разметки:
Рассмотрим случаи, которые вызвают трудности при автоматической разработке. В корпусе будут представлены по 1-2 предложения на каждый пример.
- омонимия разного рода: 
    - существительное vs глагол *печь, ели* 
    - причастия vs прилагательные *вареный, рассенный*
    - наречия vs предлоги: *вблизи*
- слова с дефисами. 
    - Не хотим делить: 
         - предлоги: *из-за* 
         - наречия: *по-настоящему*
         - названия городов, в том числе другие части речи, образованные от их названий: *Нью-Йорк*, *Йошкар-Олинский*
     - Хотим делить:
         - частицы: *-таки*
         - маршрут: *Белорецк-Магнитогорск*
         - диапазон:*5-6*
- имена собственные, как экзотические, так и фамилии на *-ов, -их*: *Тарасевича (Тарасевич), Петровский, Лидваль, Шемякин, Резвых, от Гомера до Андре Жида*
- сокращения *тыс., ред.*

* Категория "*Редкие слова*" по статье:
    * слова с неизвестным словарю корнем, но образованные с помощью продуктивных аффиксов. В моем корпусе: *симпим (симпить), заспавнилась, коупила (коупить), инфодамплю, галюники, панчи, ремув, на самике (самик), фички*
    * совсем непонятные слова. В моем корпусе: *на бэбича (бэбич), упртст, делюлю (прилагательное)*
    * окказиональные формы: *стригя, пья, победу, льзя*
    * аббривеатуры: *МГУ, ВМО, РФФИ, РАН*

Добавлю от себя, что также в соврменных текстах часто встречаются упрощенные формы местоимения, наречий: *покашт, ниче, ченить*. Их тоже хорошо бы размечать, потому что они так же часто будут встречаться в отзывах

## Словарь универсальных тегов
В рассмотрении будут **Natasha, Stanza, Mystem**

Natasha и Stanza используют Upos, у Mystem свои теги. Сравним, как это будет работать для разных частей речи и подберем один враиант

| часть речи | тег Mystem | Upos | Наш итог |
|:----------:|:----------:|:---------:|:--------:|
| прилагательное | A | ADJ | **ADJ** |
| наречие | ADV | ADV | **ADV** |
| местоименное наречие | ADVPRO | ADV | **ADV** |
| числительное-прилагательное | ANUM | ADJ | **ADJ** |
| местоимение-прилагательное | APRO | DET | **DET** |
| часть композита - сложного слова | COM | ? | **NOUN** |
| сочинительный союз | CONJ | CCONJ | **CONJ** |
| подчинительный союз | CONJ | SCONJ | **CONJ** |
| междометие | INTJ | INTJ | **INTJ** |
| числительное | NUM | NUM | **NUM** |
| частица | PART | PART | **PART** |
| предлог| PR | ADP | **ADP** |
| существительное| S | NOUN | **NOUN** |
| местоимение-существительное| SPRO | PRON | **PRON** |
| глагол | V | VERB | **VERB** |
| вспомогательный глагол | V | AUX | **VERB**
| имя собственное | S | PROPN | **NOUN** |
| пунктуация | - | PUNCT | **NA** |
| символ | - | SYM | **NA** |



Итог: в основном теги в Stanza и Natasha останутся нетронутыми. Имена собственные вспомогательные глаголы и союзы будут более обощены, а символы и пункутуация вовсе не будут учитываться.


In [2]:
tokens_frame = pd.DataFrame(columns = ['Токен', 'Тег_1', 'Тег_2']) # второй тег для неоднозначных по моему мнению случаев

Использую токенизацию с помощью nltk для удобства и превращаю корпус в таблицу. Далее надо разметить вручную все токены, проверить правильность токенизации

In [3]:
with open('corpora.txt', 'r', encoding='utf-8') as file:
    raw_text = file.read()
    tokens_text = word_tokenize(raw_text)
    tokens_frame['Токен'] = tokens_text

In [4]:
tokens_frame.to_csv('empty_tokens.csv')

In [5]:
# подгружаю размеченные вручную токены
gold_taggs = pd.read_csv('tagged.csv')

In [6]:
gold_taggs.head()

,Токен,Тег_1,Тег_2
0,Входим,VERB,NaN
1,в,ADP,NaN
2,дом,NOUN,NaN
3,",",NaN,NaN
4,открыв,VERB,NaN


In [7]:
print('Размер корпуса: ', len(gold_taggs))

Размер корпуса:  625


## Задаем функции

In [8]:
upos_tags = {'CCONJ': 'CONJ',
             'SCONJ': 'CONJ',
             'AUX': 'VERB',
             'PROPN': 'NOUN',
             'PUNCT': '',
             'SYM': ''
}

In [9]:
mystem_tags = {'A': 'ADJ', 
               'ADVPRO': 'ADV', 
               'ANUM': 'ADJ', 
               'APRO': 'DET', 
               'COM': 'NOUN', 
               'PR': 'ADP', 
               'S': 'NOUN',
               'SPRO': 'PRON',
               'V': 'VERB',
              }

In [10]:
def compare_right_tokens(token, tag, gold, results, gold_tags, index):
    token_gold = gold['Токен'][index]
    tag_gold_1 = gold['Тег_1'][index]
    tag_gold_2 = gold['Тег_2'][index]
    
    if tag == tag_gold_1:
        gold_tags.append(tag_gold_1)
    elif tag == tag_gold_2:
         gold_tags.append(tag_gold_2)
    else: gold_tags.append(tag_gold_1)
    results.append(tag)
    index += 1
    return index, results, gold_tags

In [11]:
'''
def compare(gold, unified_parser):
    results = []  # сюда будем писать результаты
    gold_tags = []     # сюда будем писать мои теги
    index = 0
    for token, tag, in zip(unified_parser['Токен'], unified_parser['Тег']):
        token_gold = gold['Токен'][index]
        tag_gold_1 = gold['Тег_1'][index]
        tag_gold_2 = gold['Тег_2'][index]
                                 
        # если все нормально с токинизацией
        if token == token_gold:
            res = compare_right_tokens(token, tag, gold, results, gold_tags, index)
            index = res[0] 
            results = res[1]
            gold_tags = res[2]
                                 
        # если неправильная токенизация
        else: 
            if token in token_gold: # парсер побил на токены лишний раз
                print('Парсер перебил ' + str(token_gold) + '. Выдал: ' + str(token))
                
                if token_gold.endswith(token):
                    results.append('')
                    gold_tags.append(tag_gold_1)
                    index += 1
                else: continue # без прибавления счетчика, пока не дойдем до конца токена
            
            elif token_gold in token: # парсер недобил 
                print('Парсер недобил ' + str(token_gold) + '. Выдал: ' + str(token))
                while not token.endswith(token_gold): 
                    token_gold = gold['Токен'][index]
                    index += 1
                results.append('')
                gold_tags.append(gold['Тег_1'][index])
                
            else: # mystem пропускает некоторые токены, например, знаки препинания
                index += 1
                res = compare_right_tokens(token, tag, gold, results, gold_tags, index)
                index = res[0] 
                results = res[1]
                gold_tags = res[2]
                
    return accuracy_score(results, gold_tags)
    '''
        

"\ndef compare(gold, unified_parser):\n    results = []  # сюда будем писать результаты\n    gold_tags = []     # сюда будем писать мои теги\n    index = 0\n    for token, tag, in zip(unified_parser['Токен'], unified_parser['Тег']):\n        token_gold = gold['Токен'][index]\n        tag_gold_1 = gold['Тег_1'][index]\n        tag_gold_2 = gold['Тег_2'][index]\n                                 \n        # если все нормально с токинизацией\n        if token == token_gold:\n            res = compare_right_tokens(token, tag, gold, results, gold_tags, index)\n            index = res[0] \n            results = res[1]\n            gold_tags = res[2]\n                                 \n        # если неправильная токенизация\n        else: \n            if token in token_gold: # парсер побил на токены лишний раз\n                print('Парсер перебил ' + str(token_gold) + '. Выдал: ' + str(token))\n                \n                if token_gold.endswith(token):\n                    results.ap

In [12]:
def compare(gold, unified_parser):
    results = []  # сюда будем писать результаты
    gold_tags = []     # сюда будем писать мои теги
    index = 0
    for token, tag, in zip(unified_parser['Токен'], unified_parser['Тег']):
        token_gold = gold['Токен'][index]
        tag_gold_1 = gold['Тег_1'][index]
        tag_gold_2 = gold['Тег_2'][index]
                                 
        # если все нормально с токинизацией
        if token == token_gold:
            res = compare_right_tokens(token, tag, gold, results, gold_tags, index)
            index = res[0] 
            results = res[1]
            gold_tags = res[2]
                                 
        # если неправильная токенизация
        else: 
            if token in str(token_gold): # парсер побил на токены лишний раз
                print('Парсер перебил ' + str(token_gold) + '. Выдал: ' + str(token))
                
                if token_gold.endswith(token):
                    results.append('')
                    gold_tags.append(tag_gold_1)
                    index += 1
                else: continue # без прибавления счетчика, пока не дойдем до конца токена
            
            elif str(token_gold) in token: # парсер недобил 
                print('Парсер недобил ' + str(token_gold) + '. Выдал: ' + str(token))
                while not token.endswith(token_gold): 
                    token_gold = gold['Токен'][index]
                    index += 1
                results.append('')
                gold_tags.append(gold['Тег_1'][index])
                
            else: 
                print('Не совпадают:', token_gold, token)
                
    return accuracy_score(results, gold_tags)

In [13]:
# создаем таблицу с разметкой теггера
def create_tagged_frame(tag_dict):
    new_frame = pd.DataFrame(columns = ['Токен', 'Тег'], index = range(0, len(tag_dict)))
    tokens = [token[0] for token in tag_dict]
    tags = [tag[1] for tag in tag_dict]
    new_frame['Токен'] = tokens
    new_frame['Тег'] = tags
    return new_frame

In [14]:
def unification(dct):
    unified = []
    for token_n_tag in dct:
        token = token_n_tag[0]
        tag = token_n_tag[1]
        if tag in mystem_tags:
            unified.append((token, mystem_tags[tag]))
        elif tag in upos_tags:
            unified.append((token, upos_tags[tag]))
        else: unified.append((token, tag))
    return create_tagged_frame(unified)
        

# Stanza

In [15]:
%%time
stanza_dict = [] # на самом деле не слвоарь, а лист коретежей, чтобы значения могли повторяться
analys = stnz(raw_text).to_dict()
for i in range(0, len(analys)):
    for sent in analys[i]:
        stanza_dict.append((sent['text'], sent['upos']))

CPU times: total: 1min
Wall time: 11.2 s


In [16]:
stanza_dict[:10]

[('Входим', 'VERB'),
 ('в', 'ADP'),
 ('дом', 'NOUN'),
 (',', 'PUNCT'),
 ('открыв', 'VERB'),
 ('дверь', 'NOUN'),
 (',', 'PUNCT'),
 ('попадаем', 'VERB'),
 ('в', 'ADP'),
 ('кухню', 'NOUN')]

In [17]:
stanza_frame = unification(stanza_dict)

In [18]:
stanza_frame.head()

,Токен,Тег
0,Входим,VERB
1,в,ADP
2,дом,NOUN
3,",",
4,открыв,VERB


In [19]:
compare(gold_taggs, stanza_frame)

Парсер недобил Немцы. Выдал: Немцы-таки
Парсер недобил Белорецк. Выдал: Белорецк-Магнитогорск
Парсер перебил Йошкар-Олинском. Выдал: Йошкар-
Парсер перебил Йошкар-Олинском. Выдал: Олинском
Парсер недобил Прим. Выдал: Прим.


0.7607260726072608

# Natasha

In [20]:
%%time
natasha_dict = [] # на самом деле не слвоарь, а лист коретежей, чтобы значения могли повторяться
doc = Doc(raw_text)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
    natasha_dict.append((token.text, token.pos))

CPU times: total: 125 ms
Wall time: 82.3 ms


In [21]:
natasha_dict[:10]

[('Входим', 'VERB'),
 ('в', 'ADP'),
 ('дом', 'NOUN'),
 (',', 'PUNCT'),
 ('открыв', 'VERB'),
 ('дверь', 'NOUN'),
 (',', 'PUNCT'),
 ('попадаем', 'VERB'),
 ('в', 'ADP'),
 ('кухню', 'NOUN')]

In [22]:
natasha_frame = unification(natasha_dict)

In [23]:
natasha_frame.head()

,Токен,Тег
0,Входим,VERB
1,в,ADP
2,дом,NOUN
3,",",
4,открыв,VERB


In [24]:
compare(gold_taggs, natasha_frame)

Парсер недобил 5. Выдал: 5-6
Парсер недобил Немцы. Выдал: Немцы-таки
Парсер перебил А.. Выдал: А
Парсер перебил А.. Выдал: .
Парсер перебил Е.. Выдал: Е
Парсер перебил Е.. Выдал: .
Парсер недобил Белорецк. Выдал: Белорецк-Магнитогорск
Парсер недобил 2001. Выдал: 2001-2002


0.7479270315091211

# Mystem

In [25]:
%%time
doc = m.analyze(raw_text)

CPU times: total: 0 ns
Wall time: 38.4 s


Заметим, что работает в разы дольше соперниц

In [26]:
mystem_dict = [] # на самом деле не слвоарь, а лист коретежей, чтобы значения могли повторяться
for i in range(0, len(doc)):
    txt = doc[i]['text'].replace(' ', '') #приклеивает пробелы к пунктуации
    try:
        mystem_dict.append((txt, doc[i]['analysis'][0]['gr'].split('=')[0].split(',')[0]))
    except: 
        if txt.isalpha() or txt.isdigit(): #пропускает незнакомые слова, числа и пунктуацию, что плохо
            mystem_dict.append((txt, ' '))
        else: 
            if (txt in punctuation or txt[0] in punctuation) and txt != '':
                mystem_dict.append((txt, ' '))

In [27]:
mystem_dict[:10]

[('Входим', 'V'),
 ('в', 'PR'),
 ('дом', 'S'),
 (',', ' '),
 ('открыв', 'V'),
 ('дверь', 'S'),
 (',', ' '),
 ('попадаем', 'V'),
 ('в', 'PR'),
 ('кухню', 'S')]

In [28]:
mystem_frame = unification(mystem_dict)

In [29]:
compare(gold_taggs, mystem_frame)

Парсер перебил А.. Выдал: А
Парсер перебил А.. Выдал: .
Парсер перебил Е.. Выдал: Е
Парсер перебил Е.. Выдал: .
Парсер перебил 15,5. Выдал: 15
Парсер перебил 15,5. Выдал: ,
Парсер перебил 15,5. Выдал: 5
Парсер недобил .. Выдал: .):


0.7454844006568144

### Итог
Лучшая точность у **Stanza**

# Применяем к анализу тональности

In [30]:
with open('reviews.json') as json_file:
    data = json.load(json_file)

In [31]:
with open('test_reviews.json') as json_file:
    test_data = json.load(json_file)

## Какие паттерны берем?

* **не + ADJ** и **не + ADV**. Могут быть сочетания типа *не плохо работает* или *далеко не лучший кинтеатр*

* **ADV + ADJ**. Интесификаторы, типа *невероятно ужасный*, могут быть одними и теми же в отзывах обоих тональностей, хотя сами по себе несут положительную семантику: *Сайт работает невероятно!*

* **ADJ + "качество"** и **ADV + "реклама"**. Судя по словарю из предыдущей домашки, слова *качество* и *реклама* довольно частотны в словаре до удаления повторов. Есть смысл посмотреть на его сочетания с разными прилагательными (*отличное качетсво, ужасное качество*) и наречиями (*мало рекламы*, *много рекламы*)

Освежим в памяти прошлый код, но теперь лемматизация будет с помощью Stanza

In [32]:
def preprocess_and_count(data):
    lemmas_count = Counter()
    for text in data:
        doc = stnz(text).to_dict()
        for sent in doc:
            for word in sent:
                lemma = word['lemma']
                if lemma.isalpha():
                    lemmas_count.update([lemma])
    return(lemmas_count)

In [33]:
%%time
good_lemmas_count = preprocess_and_count(data['good'])

CPU times: total: 13min 58s
Wall time: 2min 50s


In [34]:
%%time
bad_lemmas_count = preprocess_and_count(data['bad'])

CPU times: total: 12min 28s
Wall time: 2min 22s


In [35]:
print('Всего хороших лемм:', len(good_lemmas_count))
print('Всего плохих лемм:', len(bad_lemmas_count))

Всего хороших лемм: 1687
Всего плохих лемм: 1678


In [36]:
def deduplicate (gl, bl):
    good_lemmas_filtered = Counter()

    new_good_lemmas = {key: value for key, value in dict(gl).items()}
    new_bad_lemmas = {key: value for key, value in dict(bl).items()}
    
    for key, value in new_good_lemmas.items():
        if key in new_bad_lemmas:
            del new_bad_lemmas[key]
        else:
            good_lemmas_filtered[key] = value

    bad_lemmas_filtered = Counter(new_bad_lemmas)
    return (good_lemmas_filtered, bad_lemmas_filtered)

In [37]:
def get_freq_list(x, good, bad):
    res = deduplicate(good, bad)
    good_dict = res[0]
    bad_dict = res[1]
    freq_list = {'good': {}, 'bad': {}}
    for word, score in good_dict.items():
        if score > x:
            freq_list['good'][word] = score
    for word, score in bad_dict.items():
        if score > x:
            freq_list['bad'][word] = score
    return freq_list

In [38]:
freq_list_simple_1 = get_freq_list(1, good_lemmas_count, bad_lemmas_count) 

In [39]:
freq_list_simple_0 = get_freq_list(0, good_lemmas_count, bad_lemmas_count) 

In [40]:
def detect(freq_lists, text):
    counts = Counter({'good': 0, 'bad': 0})
    for word in text.split():
        if word in freq_lists['good']:
            counts['good'] += int(freq_lists['good'][word])
        elif word in freq_lists['bad']:
            counts['bad'] += int(freq_lists['bad'][word])
    return counts.most_common(1)[0][0]

In [41]:
# функция, которая только токенизирует слова, приводит к нижнему регистру, начальной форме
def preprocess(texts):
    lemmas = set()
    for text in texts:
        lemmas_txt = ''
        doc = stnz(text).to_dict()
        for sent in doc:
            for word in sent:
                lemma = word['lemma']
                if lemma.isalpha():
                    lemmas_txt += lemma + " "
        lemmas.add(lemmas_txt)
    return lemmas

In [42]:
def test_detect(good_test, bad_test, freq_lists):
    results = []  # сюда будем писать результаты
    gold = []     # сюда будем писать исходную тональность
    for review in good_test:
        predicted_mood = detect(freq_lists, review)
        results.append(predicted_mood)
        gold.append('good')
    for review in bad_test:
        predicted_mood = detect(freq_lists, review)
        results.append(predicted_mood)
        gold.append('bad')
    return accuracy_score(results, gold)

In [43]:
test_good = preprocess(test_data['good'])
test_bad = preprocess(test_data['bad'])

In [44]:
test_detect(test_good, test_bad, freq_list_simple_1)

0.7

In [45]:
test_detect(test_good, test_bad, freq_list_simple_0)

0.75

И вот теперь...

In [46]:
chunks_all = ['ADV ADJ', 'не ADJ', 'не ADV', 'ADJ качество', 'ADV реклама', 'качество ADJ']

In [47]:
# подсчет частотности + чанки
def preprocess_and_count_chunks(data, chunks):
    previous_pos = ''
    previous_lemma = ''
    
    lemmas_count = Counter()
    for text in data:
        doc = stnz(text).to_dict()
        for sent in doc:
            for word in sent:
                lemma = word['lemma']
                pos = word['upos']
                if lemma.isalpha():
                    if (previous_pos + ' ' + pos) in chunks:
                        lemmas_count.update([str(previous_lemma + ' ' + lemma)])
                        print(previous_lemma + ' ' + lemma)
                        
                    elif (previous_pos + ' ' + lemma) in chunks:
                        lemmas_count.update([str(previous_lemma + ' ' + lemma)])
                        print(previous_lemma + ' ' + lemma)
                        
                    elif (previous_lemma + ' ' + pos) in chunks:
                        lemmas_count.update([str(previous_lemma + ' ' + lemma)])
                        print(previous_lemma + ' ' + lemma)
                    else: lemmas_count.update([lemma])
                previous_pos = pos
                previous_lemma = lemma
                    
    return lemmas_count

In [48]:
%%time
good_chunks_count = preprocess_and_count_chunks(data['good'], chunks_all)

примерно одинаковый
лучший качество
наилучший качество
очень супирй
отличный качество
вполне достаточный
не полностью
не интересный
много хороший
не нужный
более удобный
не менее
не маленький
уже хороший
не удобно
довольно интересный
не очень
очень опытный
много хороший
очень удобный
много интересный
отличный качество
очень дорогой
особенно советский
очень удивленный
не так
очень маленький
так доступный
не надо
отличный качество
очень хороший
очень интересный
очень отличный
не удобный
очень удобный
качество отличный
интуитивно понятный
много интересный
не очень
очень удобный
достаточно старый
очень странный
много платный
много платный
не менее
очень хороший
очень удобный
очень удобный
более длительный
сейчас разнообразный
очень распространенный
не так
достаточно большой
серийно очередной
несколько затруднительный
не качественный
высокий качество
очень довольный
довольный качество
интуитивно понятный
максимально простой
действительно самый
приятно удивленный
высокий качество
качество ос

In [49]:
%%time
bad_chunks_count = preprocess_and_count_chunks(data['bad'], chunks_all)

не удобный
не адекватный
много реклама
не так
не доступный
хороший качество
достаточно скудный
более приятный
не кликабельный
более доступный
очень неудобный
очень небольшой
очень старый
очень странный
не надо
не густой
не руский
не лестный
хороший качество
не менее
менее дорогой
очень довольный
удачно целый
желательно бесплатный
не должен
где нужный
уже достаточный
там сплошной
однако уверенный
не технический
столько сложный
более удобный
сейчас нужный
столько лишний
не нужный
абсолютно невнятный
абсолютно бесполезный
совсем пыльный
хороший качество
очень отстойный
не обязательный
не самый
не приятный
срочно блокируйтый
совершенно ужасный
не так
не тогда
можно разный
так приятный
невероятный обширный
не голливудский
настолько значительный
крайне редкий
не оттуда
не возможный
много хороший
очень старый
очень осторожный
не приятный
не правда
качество пробный
внезапно невозможный
не удобный
крайне осторожный
не даром
лучший качество
не удобный
не удобный
естественно второй
не иначе
абсол

Собранные биграммы выглядят осмысленно

In [50]:
# по предыдущему оптыу, без фильтрации по частотности выходит лучше
freq_list_chunks = get_freq_list(0, good_chunks_count, bad_chunks_count) 
test_detect(test_good, test_bad, freq_list_chunks)

0.75

Качество не улучшилось, но возможно на большем количестве материала выделение чанков даст результат